# PeakAnalyzer
#### by Jan A. Meissner

Set paths for your script and the resource folder. The folder should contain all the spectral .txt files (for example: resources/CW_16_TE2A_20231031_Absorbance_12-47-13-326.txt)

In [8]:
import pandas as pd
from os import path
import os
#set current directory and directory to resources
current_directory = os.getcwd()
resource_path = os.path.join(current_directory, 'resources')

Load all data in the resource folder to a pandas object

In [15]:
files = [f for f in os.listdir(resource_path) if path.isfile(path.join(resource_path, f))]
dataset = {}
for file in files:
    file_path = os.path.join(resource_path, file)
    data = pd.read_csv(
        file_path, 
        sep='\t',              # Tab-separated values
        decimal=',',          # Comma as decimal separator
        skiprows=13           # Skip rows until the actual data starts (adjust the number as needed)
    )
    #give the columns names wavelenght and intensity
    data.columns = ["wavelength", "intensity"]  
    dataset[file] = data

print(dataset)


{'CW_16_TE2A_20231031_Absorbance_12-47-13-326.txt':       wavelength  intensity
0        341.293  -0.062994
1        341.677   0.015895
2        342.062   0.120269
3        342.446  -0.063835
4        342.830   0.845930
...          ...        ...
2042    1021.669   0.166159
2043    1021.946   0.121155
2044    1022.223   0.102370
2045    1022.501   0.121902
2046    1022.778   0.017362

[2047 rows x 2 columns]}


#### Display graph 